# Questions for prof:

1. Business case
2. Structure: created pipeline w/ flags and variables, then created function to call pipeline builder and evaluate
3. What type of model to use? How many steps? Try multiple models, # and type of steps? 
4. How to evaluate? 

combine neutral and negative to make a binary scenario

apple to build a product, wants to highlight positive tweets for marketing
or highlighting certain ones for later analysis to see what people like about this


choose a model type, baseline, 

80, 16, 4 - train, val, test 

decision tree, canon, 



## Business Understanding
A new stealth tech company is building a new product. They have hired me to gather the positive comments made on Twitter about Google and Apple products to help them understand people's favorite aspects of those products, so they can incorporate them into their new product. The tool will flag future tweets as either positive or non-positive (could be neutral or negative) for deeper analysis. The company has stated that they would like to flag as many positive tweets as possible while minimizing false positives (tweets incorrectly labeled as positive).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier
from keras import metrics
import tensorflow as tf
from tensorflow.keras import backend as K

from nltk.corpus import stopwords
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
import nltk
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger")


import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('judge-1377884607_tweet_product_company.csv', encoding='latin1')
df.info()

In [ ]:
df.dropna(subset=['tweet_text'], inplace=True)
df.drop_duplicates(subset=['tweet_text'], inplace=True)
df

In [ ]:
# Combine positive and neutral flags and remove "I can't tell"

neg_neut_combine = df[df['is_there_an_emotion_directed_at_a_brand_or_product'] != "I can't tell"]
neg_neut_combine['pos_flag'] = 0
neg_neut_combine.loc[df['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Positive emotion', 'pos_flag'] = 1


In [ ]:
neg_neut_combine

In [ ]:
total_emo_counts = neg_neut_combine['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()


In [ ]:
2968/(5372+2968+569)

In [ ]:
categories = ['No emotion', 'Positive', 'Negative']
total_emo_counts = neg_neut_combine['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()
plt.bar(categories, neg_neut_emo_counts)

# Add labels and title
plt.ylabel('Count')
plt.title('Emotion Counts')

# Show plot
plt.show()

In [ ]:
neg_neut_combine[neg_neut_combine['emotion_in_tweet_is_directed_at'].isnull()]['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

In [ ]:
categories = ['No emotion / Negative', 'Positive']
neg_neut_emo_counts = neg_neut_combine['pos_flag'].value_counts()
plt.bar(categories, neg_neut_emo_counts)

# Add labels and title
plt.ylabel('Count')
plt.title('Emotion Counts')

# Show plot
plt.show()

pos_neg_df = df[(df['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Positive emotion') | (df['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Negative emotion')]

In [ ]:
neg_neut_combine_tweets = neg_neut_combine['tweet_text']

In [ ]:
neg_neut_combine_sentiment = neg_neut_combine['pos_flag'].copy()

In [ ]:
full_sentiment_count = neg_neut_combine_sentiment.value_counts()
print(f'Sentiment count: \n{full_sentiment_count}')
print(f'Positive: {round(full_sentiment_count[0]/sum(full_sentiment_count)*100,1)}%')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(neg_neut_combine_tweets,neg_neut_combine_sentiment, 
                                                    test_size=0.15, 
                                                    random_state=42, 
                                                    stratify=neg_neut_combine_sentiment)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train_baseline = X_train[:int(len(X_train) * 0.8)]
y_train_baseline = y_train[:int(len(y_train) * 0.8)]
X_val_baseline = X_train[-int(len(X_train) * 0.2):]
y_val_baseline = y_train[-int(len(y_train) * 0.2):]

In [ ]:
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('lr', LogisticRegression(max_iter=1000))
])

# Train
clf.fit(X_train_baseline, y_train_baseline)

y_pred = clf.predict(X_val_baseline)
print(confusion_matrix(y_pred, y_val_baseline))

coef = classifier.coef_[0]
top_pos_idx = coef.argsort()[-10:][::-1]  # most positive
top_neg_idx = coef.argsort()[:10]         # most negative


print("Top words for positive class:", feature_names[top_pos_idx])
print("Top words for negative/neutral class:", feature_names[top_neg_idx])

In [ ]:
# Evaluate
ave_type = 'macro'
baseline_accuracy = accuracy_score(y_pred, y_val_baseline)
baseline_f1 = f1_score(y_pred, y_val_baseline, average=ave_type)
print(baseline_accuracy)
print(baseline_f1)

In [ ]:
# Calculate comparison score based on always guessing majority class (negative/neutral)
y_pred2 = [0]*len(y_val_baseline)
majority_guess_f1 = f1_score(y_val_baseline, y_pred2, average=ave_type)
majority_guess_acc = accuracy_score(y_val_baseline, y_pred2)

categories = ['Always Guess Negative/Neutral', 'Logistic Regression Model']
f1s = [majority_guess_f1,baseline_f1]
plt.bar(categories, f1s)

# Add labels and title
plt.ylim(0, 0.8)
plt.ylabel('F1 score')
plt.title('F1 Scores')

# Show values on top of bars (optional)
for i, count in enumerate(f1s):
    plt.text(i, count+.02, str(round(count,3)), ha='center')

# Show plot
plt.show()

In [ ]:
def run_pipe(X,y, num_words=2000, 
             remove_stopwords=False, 
             lemmatize=False, 
             num_epochs=200, 
             show_results=False, 
             balance_classes=False):

    # Compute weights to balance positive and negative comments to combat data imbalance
    classes = np.unique(y)
    class_weights = [1]*len(classes)
    if balance_classes:
        class_weights = compute_class_weight(
            class_weight="balanced",
            classes=classes,
            y=y)
#         if len(classes) == 3:
#             class_weights=[1/.15,1/.6,1/.4]

    class_weights = dict(zip(classes,class_weights))

    # ---------------------------
    # Step 1: Custom text prep transformer
    # ---------------------------

    class TextPrep(BaseEstimator, TransformerMixin):
        def __init__(self):
            self.remove_stopwords = remove_stopwords
            self.lemmatize = lemmatize
            self.stop_words = set(stopwords.words("english"))
            self.lemmatizer = WordNetLemmatizer()

        def get_wordnet_pos(self, treebank_tag):
            """Map POS tag to WordNet format for better lemmatization"""
            if treebank_tag.startswith("J"):
                return wordnet.ADJ
            elif treebank_tag.startswith("V"):
                return wordnet.VERB
            elif treebank_tag.startswith("N"):
                return wordnet.NOUN
            elif treebank_tag.startswith("R"):
                return wordnet.ADV
            else:
                return wordnet.NOUN  # fallback

        def fit(self, X, y=None):
            return self

        def transform(self, X):
            if isinstance(X, np.ndarray):
                X = X.ravel()
            X = [str(x) for x in X]

            processed = []
            for text in X:
                words = word_tokenize(text)

                # Remove stopwords if enabled
                if self.remove_stopwords:
                    words = [w for w in words if w.lower() not in self.stop_words]

                # Lemmatization if enabled
                if self.lemmatize:
                    pos_tags = pos_tag(words)
                    words = [self.lemmatizer.lemmatize(w, self.get_wordnet_pos(tag)) 
                             for w, tag in pos_tags]
                processed.append(" ".join(words))

            return processed        
        
    # ---------------------------
    # Step 2: Wrap tokenizer
    # ---------------------------
    class KerasTokenizer(BaseEstimator, TransformerMixin):
        def __init__(self, mode="binary"):
            self.num_words = num_words
            self.mode = mode
            self.tokenizer = Tokenizer(num_words=self.num_words)

        def fit(self, X, y=None):
            self.tokenizer.fit_on_texts(X)
            return self

        def transform(self, X):
            return self.tokenizer.texts_to_matrix(X, mode=self.mode)    
      
    # ---------------------------
    # Step 3: Build model
    # ---------------------------
    def build_model(input_dim, n_classes):
        model = Sequential()
        model.add(Dense(50, activation="relu", input_shape=(input_dim,)))
        model.add(Dense(25, activation="relu"))
        model.add(Dense(n_classes, activation="softmax"))
        model.compile(
            optimizer="adam",
            loss="sparse_categorical_crossentropy",
            metrics=['accuracy'],
            weighted_metrics=[]
        )
        return model

    # ---------------------------
    # Step 4: Setup pipeline
    # ---------------------------
    n_classes = len(np.unique(y))

    pipeline = Pipeline([
        ("prep", TextPrep()),
        ("tok", KerasTokenizer()),
        ("clf", KerasClassifier(
            build_fn=lambda: build_model(
                input_dim=pipeline.named_steps["tok"].num_words, 
                n_classes=n_classes),
            epochs=num_epochs,
            batch_size=256,
            verbose=0,
            validation_split=0.2, 
            callbacks=[EarlyStopping(monitor="val_accuracy", patience=8, restore_best_weights=True)],
            class_weight=class_weights)
        )
    ])
    
    # ---------------------------
    # Step 5: Fit
    # ---------------------------
    pipeline.fit(X, y)
        
    # ---------------------------
    # Step 6: Test results
    # ---------------------------   
    
    # Access history directly (no `.history`)
    pipe_history = pipeline.named_steps["clf"].history_

#     test_acc = pipeline.score(X_test, y_test)
#     print("Test Accuracy:", test_acc)

    X_val = X[-int(len(X) * 0.2):]
    y_val = y[-int(len(y) * 0.2):]

    # Get predictions and F1 score
    y_pred = pipeline.predict(X_val)
    pipe_f1 = f1_score(y_val, y_pred, average=ave_type)
#     print(y_val.value_counts())
#     print('\nPredictions:')
#     display(pd.DataFrame(y_pred).value_counts())

    # Detailed classification report
#     print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_val, y_pred))
    
    if show_results:
        # Confusion matrix
        print(confusion_matrix(y_val, y_pred))
        # Plot accuracy
        print("Max Val Accuracy:", max(pipe_history["val_accuracy"]), accuracy_score(y_val, y_pred))
        print("F1 score: ", pipe_f1)
        plt.figure(figsize=(8,5))
        plt.plot(pipe_history["accuracy"], label="Train Accuracy")
        if "val_accuracy" in pipe_history:
            plt.plot(pipe_history["val_accuracy"], label="Validation Accuracy")
        plt.title("Epoch vs Accuracy")
        plt.xlabel("Epoch")
        plt.ylabel("Accuracy")
        plt.legend()
        plt.show()
    
    return max(pipe_history["val_accuracy"]), pipe_f1;

In [ ]:
def parameter_check (X_par, y_par, num_words_list, plot=True, balance_classes=False, show_results=False):
    
    results=pd.DataFrame(columns=["number_of_words","val_accuracy", 'remove_stopwords', 'lemmatize', 'f1'])
    i=1
    stop_tf=False
    lem_tf=False
#     for j in range(3):
    for x in num_words_list:
        for stop_tf in [False,True]:
            for lem_tf in [False,True]:
                acc, f1=run_pipe(X_par,y_par, 
                                 num_words=x,
                                 remove_stopwords=stop_tf, 
                                 lemmatize=lem_tf, 
                                 balance_classes=balance_classes, 
                                 show_results=show_results)

                print(f'({i} of {len(num_words_list)*4}) Stopwords: {stop_tf}, Lemmatize: {lem_tf}, Number of words: {x}, Val accuracy: {round(acc,4)}, F1 score: {round(f1,4)}')
                new_row_data = {'number_of_words': [x], 'val_accuracy': [acc], 'remove_stopwords': [stop_tf], 'lemmatize': [lem_tf], 'f1': [f1]}
                new_row = pd.DataFrame(new_row_data)
                results = pd.concat([results,new_row], ignore_index=True)
                i += 1
    results["config"] = results.apply(
        lambda row: f"Stopwords={row['remove_stopwords']}, Lemmatize={row['lemmatize']}", axis=1)

    # Add baseline f1 score (based on always guessing neutral/positive)
#     y_val2 = y_par[-int(len(y_par) * 0.2):]
#     y_pred2 = [0]*len(y_val2)
#     baseline_f1 = f1_score(y_val2, y_pred2, average=ave_type)
#     baseline_acc = accuracy_score(y_val2, y_pred2)
    val_sentiment_count = y_train[-int(len(y_train) * 0.2):].value_counts()
    print(f'Sentiment count: \n{val_sentiment_count}')
    val_sentiment_count[0]/sum(val_sentiment_count)

    for num_word in num_words_list:
        baseline_row_data = {'number_of_words': [num_word], 'val_accuracy': [baseline_accuracy], 'f1': [baseline_f1], 'config': 'Baseline Linear Regression Model'}
        baseline_rows = pd.DataFrame(baseline_row_data)
        results = pd.concat([results,baseline_rows], ignore_index=True)
    
    results = results.sort_values("number_of_words")

    if plot:
        fig, ax1 = plt.subplots(figsize=(10,6))
        sns.lineplot(
            data=results,
            x="number_of_words",
            y="val_accuracy",
            hue="config",
            style="config",
            ax=ax1,
            markers=True,
#             estimator=None,
            dashes=False)
        
        plt.title("Validation Accuracy vs Number of Words", fontsize=14)
        plt.xlabel("Number of Words", fontsize=12)
        plt.ylabel("Validation Accuracy", fontsize=12)
        plt.legend(title="Preprocessing Config")
        plt.grid(True, alpha=0.3)
        plt.show()
        
        fig, ax2 = plt.subplots(figsize=(10,6))
        
        sns.lineplot(
            data=results,
            x="number_of_words",
            y="f1",
            hue="config",
            style="config",
            ax=ax2,
            markers=True,
            dashes=True)
        plt.title("Validation F1 Score vs Number of Words", fontsize=14)
        plt.xlabel("Number of Words", fontsize=12)
        plt.ylabel("Validation F1 Score", fontsize=12)
        plt.legend(title="Preprocessing Config")
        plt.grid(True, alpha=0.3)
        plt.show()
        
    return results

In [ ]:
test_sentiment_count = y_test.value_counts()
print(f'Sentiment count: \n{test_sentiment_count}')
test_sentiment_count[0]/sum(test_sentiment_count)

In [ ]:
val_sentiment_count = y_train[-int(len(y_train) * 0.2):].value_counts()
print(f'Sentiment count: \n{val_sentiment_count}')
val_sentiment_count[0]/sum(val_sentiment_count)

In [ ]:
num_words_list = [300, 1000, 2000, 5000, 10000]
results = parameter_check(X_train,y_train, num_words_list, show_results=False)
display(results)

In [ ]:
num_words_list = [300, 1000, 2000, 5000, 10000]
results = parameter_check(X_train,y_train, num_words_list, balance_classes=True, show_results=False)

In [ ]:
results

In [ ]:
y_val2 = y_train[-int(len(y_train) * 0.2):]

# Get predictions and F1 score
y_pred2 = [0]*len(y_val2)
f1_score(y_val2, y_pred2, average='weighted')

In [ ]:
# run_pipe(X_train,y_train, num_words=500, num_epochs=300, remove_stopwords=False, lemmatize=True)

# don't use X_test, y_test in final version. print val results

### add neutral/can't tell

rand_df = df[df['is_there_an_emotion_directed_at_a_brand_or_product'] != "I can't tell"].sample(frac=1, random_state=42).reset_index(drop=True)

rand_df['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

tweets = rand_df['tweet_text']
sentiment = rand_df['is_there_an_emotion_directed_at_a_brand_or_product'].copy()

X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(tweets,sentiment, test_size=0.15, random_state=7, stratify=sentiment)
# X_val_all = X_train_all[-int(len(X_train_all) * 0.2):]
# y_val_all = y_train_all[-int(len(y_train_all) * 0.2):]

round(sentiment.value_counts()/len(sentiment)*100,1)

round(y_train_all.value_counts()/len(y_train_all)*100,1)

round(y_test_all.value_counts()/len(y_test_all)*100,1)

y_val_all = y_train_all[-int(len(y_train_all) * 0.2):]
round(y_val_all.value_counts()/len(y_val_all)*100,1)

num_words_list = [300, 1000, 2000, 5000, 10000]
results = parameter_check(X_train_all,y_train_all, num_words_list, balance_classes=False, show_results=False)

762/(31+71+595+55+762)

num_words_list = [300, 1000, 2000, 5000, 10000]
results = parameter_check(X_train_all,y_train_all, num_words_list, balance_classes=True, show_results=False)

y_val_all.value_counts()